# Imports, constants and functions

In [1]:
import os
import nrrd
import numpy as np
import pandas as pd
import sklearn as sk
import seaborn as sns
import pingouin as pg
import radiomics as pr
import SimpleITK as sitk
import matplotlib.pyplot as plt
from radiomics import featureextractor
from sklearn.linear_model import lasso_path
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from scipy.ndimage import binary_dilation, binary_erosion
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

In [40]:
# Configura pandas para mostrar todo el DataFrame
pd.set_option('display.max_rows', None)      # Todas las filas
pd.set_option('display.max_columns', None)   # Todas las columnas
pd.set_option('display.width', None)         # Todo el ancho posible
pd.set_option('display.max_colwidth', None)  # Ancho completo del contenido de la columna

In [2]:
# Constantes
INPUT_PATH = '/Users/veramegias/Documents/Universidad/Cuarto/TFG/Segmentaciones'
IMAGES_PATH = 'images'
OUTPUT_PATH = 'outputs'
REEXECUTE = True
MAX_NUM = 3

columns_file_text = f'{OUTPUT_PATH}/columnas_df_features.txt'

In [3]:
def preprocess_features(df, label_column, threshold_corr=0.95, threshold_var=0.01):
    """
    Preprocesa un DataFrame eliminando variables constantes, colineales y normalizando los datos.

    Pasos:
        1. Separa las variables independientes (X) y la variable objetivo (y).
        2. Elimina variables con baja varianza (prácticamente constantes).
        3. Elimina variables con alta correlación para reducir colinealidad.
        4. Normaliza las características con StandardScaler.

    Args:
        df (pd.DataFrame): DataFrame con las características radiómicas y la columna de la variable objetivo.
        label_column (str): Nombre de la columna que contiene la variable objetivo.
        threshold_corr (float): Umbral de correlación para eliminar variables (default 0.95).
        threshold_var (float): Umbral de varianza mínima para eliminar variables constantes (default 0.01).
    
    Returns:
        pd.DataFrame: DataFrame con las características preprocesadas, filtradas y normalizadas.
        pd.Series: Variable objetivo (y).
        list: Lista de variables eliminadas.
    """
    n_variables = len(df.columns)
    # 1. Separar la variable objetivo (y) de las características (X)
    X = df.drop(columns=[label_column])  # Todas las columnas excepto la de la etiqueta
    y = df[label_column]                 # Columna de la variable objetivo
    
    # 2. Eliminar variables con baja varianza
    var_selector = VarianceThreshold(threshold=threshold_var)
    X_var_filtered = pd.DataFrame(var_selector.fit_transform(X), columns=X.columns[var_selector.get_support()])
    removed_low_var = list(set(X.columns) - set(X_var_filtered.columns))
    
    print(f"Se eliminaron {len(removed_low_var)} variables con baja varianza.")

    # 3. Eliminar variables altamente correlacionadas
    corr_matrix = X_var_filtered.corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    to_drop = [column for column in upper.columns if any(upper[column] > threshold_corr)]
    
    X_filtered = X_var_filtered.drop(columns=to_drop)
    
    print(f"Se eliminaron {len(to_drop)} variables altamente correlacionadas.")

    # 4. Normalizar las características con StandardScaler y devolverlo como DataFrame
    scaler = StandardScaler()
    X_scaled = pd.DataFrame(scaler.fit_transform(X_filtered), columns=X_filtered.columns, index=X.index)
    
    print(f'Se han eliminado {len(removed_low_var + to_drop)} de {n_variables} variables')

    return X_scaled, y, removed_low_var + to_drop

In [4]:
def transform_dataframe(df):
    """
    Transforma un DataFrame con celdas de diferentes clases a numeros o strings.
    
    Para columnas que contienen listas o tuplas:
    - Si la lista o tupla tiene un único elemento, convierte el valor en un número o string según corresponda.
    - Si la lista o tupla tiene múltiples elementos, expande la columna en varias columnas, 
      una por cada elemento de la lista o tupla. Las nuevas columnas se nombran usando el nombre 
      original seguido por un sufijo `_1`, `_2`, etc.

    Para columnas que contienen diccionarios:
    - Cada clave del diccionario se convierte en una nueva columna.
    - Si un valor del diccionario es un array/lista o tupla:
        - Si tiene un único elemento, se convierte en un valor único.
        - Si tiene múltiples elementos, genera columnas adicionales con sufijos `_1`, `_2`, etc.
    - Las nuevas columnas se nombran usando el nombre original seguido por `_{key}` y, si es necesario, 
      un sufijo adicional para los arrays o tuplas.
    - Elimina la columna original una vez procesada.

    Args:
        df (pd.DataFrame): DataFrame original.
    Returns:
        df (pd.DataFrame): DataFrame transformado.
    """
    # Crear una copia para no modificar el original
    transformed_df = df.copy()

    # Iterar sobre las columnas
    for col in transformed_df.columns:
        # Identificar las celdas que son listas, tuplas o arrays
        if transformed_df[col].apply(lambda x: isinstance(x, (list, tuple))).any():
            # Expandir los valores si hay listas/tuplas con más de un elemento
            expanded = transformed_df[col].apply(lambda x: list(x) if isinstance(x, (list, tuple)) else [x])
            
            # Verificar la longitud máxima de las listas/tuplas
            max_len = expanded.apply(len).max()
            
            if max_len > 1:
                # Crear nuevas columnas para listas/tuplas con múltiples elementos
                for i in range(max_len):
                    transformed_df[f"{col}_{i+1}"] = expanded.apply(lambda x: x[i] if i < len(x) else None)
                
                # Eliminar la columna original
                transformed_df.drop(columns=[col], inplace=True)
            else:
                # Convertir listas/tuplas con un único elemento en valores (número o string)
                transformed_df[col] = expanded.apply(lambda x: x[0] if len(x) == 1 else x)
        
        # Identificar las celdas que son diccionarios
        elif transformed_df[col].apply(lambda x: isinstance(x, dict)).any():
            # Expandir las claves del diccionario en nuevas columnas
            dict_expansion = transformed_df[col].apply(lambda x: x if isinstance(x, dict) else {})
            keys = set(k for d in dict_expansion for k in d.keys())
            
            for key in keys:
                # Extraer los valores de la clave específica
                key_values = dict_expansion.apply(lambda x: x.get(key, None))
                
                # Si los valores son arrays, listas o tuplas, manejarlos como tal
                if key_values.apply(lambda x: isinstance(x, (list, tuple))).any():
                    # Expandir los arrays/tuplas en columnas adicionales
                    expanded = key_values.apply(lambda x: list(x) if isinstance(x, (list, tuple)) else [x])
                    max_len = expanded.apply(len).max()
                    
                    for i in range(max_len):
                        transformed_df[f"{col}_{key}_{i+1}"] = expanded.apply(lambda x: x[i] if i < len(x) else None)
                else:
                    # Si no son listas/tuplas, mantener el valor tal cual
                    transformed_df[f"{col}_{key}"] = key_values
            
            # Eliminar la columna original
            transformed_df.drop(columns=[col], inplace=True)

    return transformed_df

In [5]:
def same_sizes(image1, image2):
    """
    Comprueba que la imagen 1 y la imagen 1 tienen las mismas dimensiones.
    Args:
        image1 (SimpleITK.Image): Imagen 1.
        image2 (SimpleITK.Image): Imagen 2.
    Returns:
        boolean: Si el tamaño coincide
    """
    return image1.GetSize() == image2.GetSize()

In [6]:
def simulate_resegmentation(mask, method='dilation', iterations=1):
    """
    Simula una segunda segmentación modificando la máscara original.
    
    Parámetros:
        mask (sitk.Image): Máscara original.
        method (str): 'dilation' o 'erosion'. Se usa para simular una resegmentación.
        iterations (int): Número de iteraciones de la operación morfológica.
    
    Retorna:
        sitk.Image: Máscara modificada.
    """
    mask_arr = sitk.GetArrayFromImage(mask)
    
    if method == 'dilation':
        mask_arr_mod = binary_dilation(mask_arr, structure=np.ones((3,3,3)), iterations=iterations)
    elif method == 'erosion':
        mask_arr_mod = binary_erosion(mask_arr, structure=np.ones((3,3,3)), iterations=iterations)
    else:
        raise ValueError("El método debe ser 'dilation' o 'erosion'")
    
    mask_mod = sitk.GetImageFromArray(mask_arr_mod.astype(np.uint8))
    mask_mod.CopyInformation(mask)
    return mask_mod

In [7]:
def simulate_interpolation(image, new_spacing):
    """
    Re-muestrea la imagen a un espaciamiento ligeramente modificado.
    
    Parámetros:
        image (sitk.Image): Imagen original.
        new_spacing (tuple): Nuevo espaciamiento (por ejemplo, (sx, sy, sz)).
    
    Retorna:
        sitk.Image: Imagen re-muestreada.
    """
    original_spacing = image.GetSpacing()
    original_size = image.GetSize()
    new_size = [int(round(osz * ospc / nspc)) for osz, ospc, nspc in zip(original_size, original_spacing, new_spacing)]
    
    resample = sitk.ResampleImageFilter()
    resample.SetOutputSpacing(new_spacing)
    resample.SetSize(new_size)
    resample.SetOutputDirection(image.GetDirection())
    resample.SetOutputOrigin(image.GetOrigin())
    resample.SetInterpolator(sitk.sitkLinear)
    new_image = resample.Execute(image)
    return new_image

In [8]:
def simulate_interpolation_mask(mask, new_spacing):
    """
    Re-muestrea la máscara (usando interpolación de vecino más cercano) a un espaciamiento modificado.
    
    Parámetros:
        mask (sitk.Image): Máscara original.
        new_spacing (tuple): Nuevo espaciamiento.
    
    Retorna:
        sitk.Image: Máscara re-muestreada.
    """
    original_spacing = mask.GetSpacing()
    original_size = mask.GetSize()
    new_size = [int(round(osz * ospc / nspc)) for osz, ospc, nspc in zip(original_size, original_spacing, new_spacing)]
    
    resample = sitk.ResampleImageFilter()
    resample.SetOutputSpacing(new_spacing)
    resample.SetSize(new_size)
    resample.SetOutputDirection(mask.GetDirection())
    resample.SetOutputOrigin(mask.GetOrigin())
    resample.SetInterpolator(sitk.sitkNearestNeighbor)
    new_mask = resample.Execute(mask)
    return new_mask

In [9]:
def get_features(image, mask):
    if same_sizes(image, mask):
        extractor = featureextractor.RadiomicsFeatureExtractor()
    else:
        print(f'[ERROR] Sizes are not the same.')
    features = extractor.execute(image, mask)
    
    return features

In [10]:
def extract_features(image_file_path, mask_file_path, process_type):
    """
    Extrae características radiómicas de una imagen y su máscara utilizando PyRadiomics.
    Args:
        image_file_path (str): Ruta al archivo NRRD que contiene la imagen a analizar.
        mask_file_path (str): Ruta al archivo NRRD que contiene la máscara asociada a la imagen.
    Returns:
        features (dict): Características radiómicas extraídas.
    Extra:
        Comprueba que el tamaño de las imagenes sea compatible.
    """
    # Load paths and images
    image_data, _ = nrrd.read(image_file_path)
    image = sitk.GetImageFromArray(image_data)
    mask_data, _ = nrrd.read(mask_file_path)
    mask = sitk.GetImageFromArray(mask_data)

    if process_type == "resegmentation":
        mask = simulate_resegmentation(mask, method='dilation', iterations=1)
    elif process_type == "interpolation":
        original_spacing = image.GetSpacing()
        new_spacing = tuple([s + 0.1 for s in original_spacing])
        image = simulate_interpolation(image, new_spacing)
        mask  = simulate_interpolation_mask(mask, new_spacing)
    elif process_type == "original":
        pass
    else:
        print("[ERROR]  No valid process. Nothing applied.")

    return get_features(image, mask)

In [34]:
def join_original_columns(df_original, df_interes):
    if len(df_original) != len(df_interes):
        raise ValueError("Los DataFrames tienen tamaños diferentes; no se pueden unir correctamente.")
    
    df_final = pd.concat([df_original.reset_index(drop=True), df_interes.reset_index(drop=True)], axis=1)

    return df_final

In [35]:
def compute_icc(df, feature_col, subject_col='subject', method_col='method'):
    """
    Calcula el ICC para una característica dada utilizando los modelos ICC2 e ICC3.
    
    Parámetros:
        df (pd.DataFrame): DataFrame con las mediciones.
        feature_col (str): Nombre de la columna que contiene la característica a evaluar.
        subject_col (str): Columna que identifica al sujeto.
        method_col (str): Columna que identifica el método o la medición.
    
    Retorna:
        dict: Diccionario con los valores de ICC2 e ICC3.
    """
    
    icc_df = pg.intraclass_corr(data=df, targets=subject_col, raters=method_col, ratings=feature_col)
    icc2 = icc_df.loc[icc_df['Type'] == 'ICC2', 'ICC'].values[0]
    icc3 = icc_df.loc[icc_df['Type'] == 'ICC3', 'ICC'].values[0]
    return {'ICC2': icc2, 'ICC3': icc3}

In [11]:
def convert_columns_to_numeric(df):
    def unpack_single_element_cells(cell):
        # Maneja arrays escalares de numpy
        if isinstance(cell, np.ndarray):
            if cell.shape == ():  # array escalar
                return cell.item()
            elif cell.size == 1:  # array con un solo elemento
                return cell[0]
        elif isinstance(cell, list):
            if len(cell) == 1:
                return cell[0]
        return cell

    numeric_df = df.copy()

    for col in numeric_df.columns:
        numeric_df[col] = numeric_df[col].apply(
            lambda x: unpack_single_element_cells(x) if isinstance(x, (list, np.ndarray)) else x
        )

        try:
            if numeric_df[col].dtype == 'bool':
                numeric_df[col] = numeric_df[col].astype(int)
            numeric_df[col] = pd.to_numeric(numeric_df[col], errors='raise')
        except Exception as e:
            print(f'[ERROR] al convertir la columna {col}: {e}')
            numeric_df.drop(columns=[col], inplace=True)

    return numeric_df


In [12]:
def plot_correlation_matrix(df):
    """
    Genera una matriz de correlación con un mapa de calor.
    
    Args:
        df (DataFrame): DataFrame con los datos.
    """
    df = df.loc[:, (df != df.iloc[0]).any()]
    correlation_matrix = df.corr()
    mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))  # Parte superior (cambiar a np.tril para inferior)

    print('Be aware that columns with constant values will not be plot.')
    plt.figure(figsize=(10, 8))
    correlation_matrix = df.corr()
    sns.heatmap(correlation_matrix, cmap='coolwarm', mask=mask)
    plt.title("Matriz de Correlación")
    plt.savefig(f'{IMAGES_PATH}/correlation_matrix.png')
    plt.show()

In [13]:
def plot_lasso_path(X, y):
    """
    Genera un gráfico del camino de Lasso para analizar la importancia de las variables.
    
    Args:
        X (array-like): Variables independientes.
        y (array-like): Variable dependiente.
    """
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X)
    y = y.values.ravel()
    print("Número de condición de X:", np.linalg.cond(X))
    alphas, coefs, _ = lasso_path(X_train_scaled, y, max_iter=10000, tol=1e-3)
    print("Condición de X:", np.linalg.cond(X_train_scaled))
    plt.figure(figsize=(10, 6))
    for coef in coefs:
        plt.plot(-np.log10(alphas), coef)
    
    plt.xlabel("-Log10(Alpha)")
    plt.ylabel("Coeficientes")
    plt.title("Lasso Path")
    plt.grid(True)
    plt.savefig(f'{IMAGES_PATH}/lasso_path.png')
    plt.show()

In [14]:
def plot_variable_distribution(df):
    """
    Genera gráficos de distribución para todas las columnas numéricas.
    
    Args:
        df (DataFrame): DataFrame con los datos.
    """
    numeric_columns = df.select_dtypes(include=['number']).columns
    for column in numeric_columns:
        plt.figure(figsize=(8, 4))
        sns.histplot(df[column], kde=True, bins=30)
        plt.title(f"Distribución de {column}")
        plt.xlabel(column)
        plt.ylabel("Frecuencia")
        plt.show()

In [15]:
def plot_feature_importance(X, y, feature_names):
    """
    Genera un gráfico de importancia de características usando un modelo de Random Forest.
    
    Args:
        X (array-like): Variables independientes.
        y (array-like): Variable dependiente.
        feature_names (list): Nombres de las características.
    """
    model = RandomForestClassifier(random_state=0)
    model.fit(X, y)
    importance = model.feature_importances_

    plt.figure(figsize=(10, 6))
    plt.barh(feature_names, importance)
    plt.xlabel("Importancia")
    plt.ylabel("Características")
    plt.title("Importancia de las Características")
    plt.savefig(f'{IMAGES_PATH}/feature_importance_RF.png')
    plt.show()

In [16]:
def calculate_vif(X):
    """
    Calcula el Factor de Inflación de la Varianza (VIF) para detectar multicolinealidad.
    
    Args:
        X (DataFrame): Variables independientes.
    """
    vif_data = pd.DataFrame()
    vif_data["Variable"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    print(vif_data)

In [17]:
def filter_columns_by_file(df, file_path):
    """
    Filtra las columnas del dataframe según un archivo con nombres de columnas
    seguidos de "OK" o "NO". Las columnas con "NO" se eliminan.
    """
    columns_to_keep = []

    with open(file_path, "r") as f:
        for line in f:
            column_info = line.strip().split()
            if len(column_info) == 2:  # Asegurarse de que hay un nombre y un estado
                column_name, status = column_info
                if status == "OK":
                    columns_to_keep.append(column_name)

    df_clean = df.copy()
    return df_clean[columns_to_keep]

In [18]:
def compute_icc(df, feature_col, subject_col='subject', method_col='method'):
    """
    Calcula el ICC para una característica dada utilizando los modelos ICC2 e ICC3.
    
    Parámetros:
        df (pd.DataFrame): DataFrame con las mediciones.
        feature_col (str): Nombre de la columna que contiene la característica a evaluar.
        subject_col (str): Columna que identifica al sujeto.
        method_col (str): Columna que identifica el método o la medición.
    
    Retorna:
        dict: Diccionario con los valores de ICC2 e ICC3.
    """
    icc_df = pg.intraclass_corr(data=df, targets=subject_col, raters=method_col, ratings=feature_col)
    icc2 = icc_df.loc[icc_df['Type'] == 'ICC2', 'ICC'].values[0]
    icc3 = icc_df.loc[icc_df['Type'] == 'ICC3', 'ICC'].values[0]
    return {'ICC2': icc2, 'ICC3': icc3}

In [43]:
def analyze_icc(df_original, df_reseg, df_interp):
    """
    Integra tres DataFrames (original, resegmentación e interpolación), 
    asigna una etiqueta a cada uno y calcula el ICC para cada característica.
    
    Parámetros:
        df_original (pd.DataFrame): DataFrame con las features de la imagen original.
        df_reseg (pd.DataFrame): DataFrame con las features tras resegmentación.
        df_interp (pd.DataFrame): DataFrame con las features tras interpolación/muestreo.
    
    Retorna:
        pd.DataFrame: DataFrame en el que cada fila corresponde a una característica
                      y se muestran los valores de ICC2 e ICC3.
    """
    df_all = pd.concat([df_original, df_reseg, df_interp], ignore_index=True)
    print('Dataframes concatenados')
    
    important_data = df_all[['subject','method']].copy()
    df_all = filter_columns_by_file(df_all, columns_file_text)
    df_all = convert_columns_to_numeric(df_all)
    df_all = join_original_columns(df_all, important_data)
    features_list = [col for col in df_all.columns if col not in ['subject', 'method']]

    results = {}
    for feat in features_list:
        results[feat] = compute_icc(df_all, feat, subject_col='subject', method_col='method')
    return pd.DataFrame.from_dict(results, orient='index')

# Load data

In [19]:
# Para cambiar los nombres de los ficheros series, ejecutar en terminal en la carpeta de inputs:
# for file in series*.nrrd; do mv "$file" "${file/series/serie}"; done

In [20]:
if REEXECUTE:
    df_features = pd.DataFrame()

    for num in range(1,MAX_NUM):
        for extension in ['oc', 'ccr']:
            if num == 36 or num == 42 and extension == 'oc':  # Excluir el fichero 36oc y 42oc
                continue
            print(f'Executing {num} for {extension}')
            image_path = os.path.join(INPUT_PATH, extension, str(num)+extension, 'Seg'+str(num)+extension, 'serie'+str(num)+extension+'.nrrd')
            mask_path = os.path.join(INPUT_PATH, extension, str(num)+extension, 'Seg'+str(num)+extension, extension+str(num)+'.nrrd')        
            if not os.path.exists(image_path):
                print(f'ERROR: No such file for {image_path}')
            if not os.path.exists(mask_path):
                print(f'ERROR: No such file for {mask_path}')
            features_image = extract_features(image_path, mask_path, "original")
            features_image['cancer'] = 'ccr' in mask_path
            features_image['subject'] = str(num)
            features_image['method'] = "original"
            df_features = pd.concat([df_features, pd.DataFrame([features_image])], ignore_index=True)
    df_features.to_csv('df_features.csv', index=False)
else:
    df_features = pd.read_csv('df_features.csv')

Executing 1 for oc


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


Executing 1 for ccr


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


Executing 2 for oc


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


Executing 2 for ccr


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


In [21]:
df_features.shape

(4, 132)

In [22]:
df_features.head()

,diagnostics_Versions_PyRadiomics,diagnostics_Versions_Numpy,diagnostics_Versions_SimpleITK,diagnostics_Versions_PyWavelet,diagnostics_Versions_Python,diagnostics_Configuration_Settings,diagnostics_Configuration_EnabledImageTypes,diagnostics_Image-original_Hash,diagnostics_Image-original_Dimensionality,diagnostics_Image-original_Spacing,...,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength,cancer,subject,method
0,v3.0.1,1.24.3,2.3.1-g42ce2,1.4.1,3.11.5,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},8d7665c7a69d5330cb6330fd4c3f96d74645bec8,3D,"(1.0, 1.0, 1.0)",...,0.13333333333333333,472.38793103448273,0.8944960934131476,0.013839926071630869,40.89113618170633,0.05260342894628025,0.5354228455612338,False,1,original
1,v3.0.1,1.24.3,2.3.1-g42ce2,1.4.1,3.11.5,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},e0ab011acfc4b85b21e77dfe6699d755d5d9c243,3D,"(1.0, 1.0, 1.0)",...,0.0238012022927443,226718.1994906859,39.25281278724457,0.00031292808962871555,32.36342742333152,0.02055317617087976,0.014939089194857856,True,1,original
2,v3.0.1,1.24.3,2.3.1-g42ce2,1.4.1,3.11.5,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},22e18047512aa52054d6739e261bd24e91937a10,3D,"(1.0, 1.0, 1.0)",...,0.023817399933840556,32596.597029320987,5.7200435491548625,0.003241768060462912,12.774190506780394,0.02383802375880444,0.08217928137111477,False,2,original
3,v3.0.1,1.24.3,2.3.1-g42ce2,1.4.1,3.11.5,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},0196941df1cf094bb18fca113a3ef8454662bc6c,3D,"(1.0, 1.0, 1.0)",...,0.017617021276595746,1430205.0723304094,216.11608883560078,9.301911345185159e-05,18.54536992978121,0.010392536561354006,0.0038335969318926645,True,2,original


# Clean dataframe

In [23]:
df_features[df_features['cancer'] == True]

,diagnostics_Versions_PyRadiomics,diagnostics_Versions_Numpy,diagnostics_Versions_SimpleITK,diagnostics_Versions_PyWavelet,diagnostics_Versions_Python,diagnostics_Configuration_Settings,diagnostics_Configuration_EnabledImageTypes,diagnostics_Image-original_Hash,diagnostics_Image-original_Dimensionality,diagnostics_Image-original_Spacing,...,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength,cancer,subject,method
1,v3.0.1,1.24.3,2.3.1-g42ce2,1.4.1,3.11.5,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},e0ab011acfc4b85b21e77dfe6699d755d5d9c243,3D,"(1.0, 1.0, 1.0)",...,0.0238012022927443,226718.1994906859,39.25281278724457,0.00031292808962871555,32.36342742333152,0.02055317617087976,0.014939089194857856,True,1,original
3,v3.0.1,1.24.3,2.3.1-g42ce2,1.4.1,3.11.5,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},0196941df1cf094bb18fca113a3ef8454662bc6c,3D,"(1.0, 1.0, 1.0)",...,0.017617021276595746,1430205.0723304094,216.11608883560078,9.301911345185159e-05,18.54536992978121,0.010392536561354006,0.0038335969318926645,True,2,original


In [24]:
if False:
    with open(f'{columns_file_text}', "w") as f:
        for column in df_features.columns:
            f.write(column + " OK \n")

### Edita el documento de columnas para eliminarlas. Pon "NO" en las que quieras eliminar.

In [25]:
df_clean = filter_columns_by_file(df_features, columns_file_text)

In [26]:
df_clean = convert_columns_to_numeric(df_clean)

[ERROR] al convertir la columna diagnostics_Image-original_Size: Invalid object type at position 0


In [27]:
df_clean.head()

,original_shape_Elongation,original_shape_Flatness,original_shape_Maximum3DDiameter,original_shape_Sphericity,original_shape_SurfaceArea,original_shape_SurfaceVolumeRatio,original_firstorder_Entropy,original_firstorder_InterquartileRange,original_firstorder_Kurtosis,original_firstorder_Maximum,...,original_glszm_SizeZoneNonUniformityNormalized,original_glszm_SmallAreaEmphasis,original_glszm_SmallAreaHighGrayLevelEmphasis,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Contrast,cancer
0,0.929791,0.222162,17.262677,0.563530,483.190076,1.143645,2.495008,41.0,4.553240,202.0,...,0.293698,0.558183,20.544141,0.064195,4.730669,4.723879e+02,0.894496,0.013840,0.052603,0
1,0.846585,0.514752,50.537115,0.853392,5296.197639,0.185362,2.473488,47.0,2.860854,169.0,...,0.277628,0.539211,22.209414,0.027652,5.531665,2.267182e+05,39.252813,0.000313,0.020553,1
2,0.843453,0.553572,25.396850,0.789201,1274.975281,0.424809,2.169235,30.0,4.503560,199.0,...,0.214120,0.457342,9.929764,0.095088,4.829599,3.259660e+04,5.720044,0.003242,0.023838,0
3,0.914844,0.554544,67.305275,0.859906,10631.268322,0.129347,1.866008,28.0,3.092082,153.0,...,0.251090,0.511141,11.133436,0.068632,5.152615,1.430205e+06,216.116089,0.000093,0.010393,1


In [28]:
columns_df_clean = df_clean.columns

# ICC process

In [ ]:
df_features_reseg = pd.DataFrame()
df_features_interp = pd.DataFrame()

for num in range(1,MAX_NUM):
    for extension in ['oc', 'ccr']:
        if num == 36 or num == 42 and extension == 'oc':  # Excluir el fichero 36oc y 42oc
            continue
        print(f'Executing {num} for {extension}')
        image_path = os.path.join(INPUT_PATH, extension, str(num)+extension, 'Seg'+str(num)+extension, 'serie'+str(num)+extension+'.nrrd')
        mask_path = os.path.join(INPUT_PATH, extension, str(num)+extension, 'Seg'+str(num)+extension, extension+str(num)+'.nrrd')

        if not os.path.exists(image_path):
            print(f'ERROR: No such file for {image_path}')
        if not os.path.exists(mask_path):
            print(f'ERROR: No such file for {mask_path}')
            
        features_reseg = extract_features(image_path, mask_path, "resegmentation")
        features_interp = extract_features(image_path, mask_path, "interpolation")
        features_reseg['cancer'] = 'ccr' in mask_path
        features_interp['cancer'] = 'ccr' in mask_path
        features_reseg['subject'] = str(num)
        features_interp['subject'] = str(num)
        features_reseg['method'] = "resegmentation"
        features_interp['method'] = "interpolation"
        df_features_reseg = pd.concat([df_features_reseg, pd.DataFrame([features_reseg])], ignore_index=True)
        df_features_interp = pd.concat([df_features_interp, pd.DataFrame([features_interp])], ignore_index=True)

Executing 1 for oc


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


Executing 1 for ccr


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


Executing 2 for oc


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


Executing 2 for ccr


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


In [30]:
print(df_features.shape)

(4, 132)


In [31]:
df_features

,diagnostics_Versions_PyRadiomics,diagnostics_Versions_Numpy,diagnostics_Versions_SimpleITK,diagnostics_Versions_PyWavelet,diagnostics_Versions_Python,diagnostics_Configuration_Settings,diagnostics_Configuration_EnabledImageTypes,diagnostics_Image-original_Hash,diagnostics_Image-original_Dimensionality,diagnostics_Image-original_Spacing,...,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength,cancer,subject,method
0,v3.0.1,1.24.3,2.3.1-g42ce2,1.4.1,3.11.5,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},8d7665c7a69d5330cb6330fd4c3f96d74645bec8,3D,"(1.0, 1.0, 1.0)",...,0.13333333333333333,472.38793103448273,0.8944960934131476,0.013839926071630869,40.89113618170633,0.05260342894628025,0.5354228455612338,False,1,original
1,v3.0.1,1.24.3,2.3.1-g42ce2,1.4.1,3.11.5,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},e0ab011acfc4b85b21e77dfe6699d755d5d9c243,3D,"(1.0, 1.0, 1.0)",...,0.0238012022927443,226718.1994906859,39.25281278724457,0.00031292808962871555,32.36342742333152,0.02055317617087976,0.014939089194857856,True,1,original
2,v3.0.1,1.24.3,2.3.1-g42ce2,1.4.1,3.11.5,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},22e18047512aa52054d6739e261bd24e91937a10,3D,"(1.0, 1.0, 1.0)",...,0.023817399933840556,32596.597029320987,5.7200435491548625,0.003241768060462912,12.774190506780394,0.02383802375880444,0.08217928137111477,False,2,original
3,v3.0.1,1.24.3,2.3.1-g42ce2,1.4.1,3.11.5,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},0196941df1cf094bb18fca113a3ef8454662bc6c,3D,"(1.0, 1.0, 1.0)",...,0.017617021276595746,1430205.0723304094,216.11608883560078,9.301911345185159e-05,18.54536992978121,0.010392536561354006,0.0038335969318926645,True,2,original


In [32]:
df_features

,diagnostics_Versions_PyRadiomics,diagnostics_Versions_Numpy,diagnostics_Versions_SimpleITK,diagnostics_Versions_PyWavelet,diagnostics_Versions_Python,diagnostics_Configuration_Settings,diagnostics_Configuration_EnabledImageTypes,diagnostics_Image-original_Hash,diagnostics_Image-original_Dimensionality,diagnostics_Image-original_Spacing,...,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength,cancer,subject,method
0,v3.0.1,1.24.3,2.3.1-g42ce2,1.4.1,3.11.5,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},8d7665c7a69d5330cb6330fd4c3f96d74645bec8,3D,"(1.0, 1.0, 1.0)",...,0.13333333333333333,472.38793103448273,0.8944960934131476,0.013839926071630869,40.89113618170633,0.05260342894628025,0.5354228455612338,False,1,original
1,v3.0.1,1.24.3,2.3.1-g42ce2,1.4.1,3.11.5,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},e0ab011acfc4b85b21e77dfe6699d755d5d9c243,3D,"(1.0, 1.0, 1.0)",...,0.0238012022927443,226718.1994906859,39.25281278724457,0.00031292808962871555,32.36342742333152,0.02055317617087976,0.014939089194857856,True,1,original
2,v3.0.1,1.24.3,2.3.1-g42ce2,1.4.1,3.11.5,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},22e18047512aa52054d6739e261bd24e91937a10,3D,"(1.0, 1.0, 1.0)",...,0.023817399933840556,32596.597029320987,5.7200435491548625,0.003241768060462912,12.774190506780394,0.02383802375880444,0.08217928137111477,False,2,original
3,v3.0.1,1.24.3,2.3.1-g42ce2,1.4.1,3.11.5,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},0196941df1cf094bb18fca113a3ef8454662bc6c,3D,"(1.0, 1.0, 1.0)",...,0.017617021276595746,1430205.0723304094,216.11608883560078,9.301911345185159e-05,18.54536992978121,0.010392536561354006,0.0038335969318926645,True,2,original


In [ ]:
print("Analizando ICCs...")
results_icc = analyze_icc(df_features, df_features_reseg, df_features_interp)
print("Resultados del ICC para cada característica:")

Analizando ICCs...
Dataframes concatenados
[ERROR] al convertir la columna diagnostics_Image-original_Size: Invalid object type at position 0


/Users/veramegias/anaconda3/lib/python3.11/site-packages/pingouin/reliability.py:367: RuntimeWarning: invalid value encountered in scalar divide
  l2 = n * (msb - f2u * mse) / (f2u * (k * msj + (k * n - k - n) * mse) + n * msb)


Resultados del ICC para cada característica:
                                      ICC2      ICC3
original_shape_Elongation         0.258850  0.998316
original_shape_Flatness           0.962607  0.988874
original_shape_Maximum3DDiameter  0.970904  0.999693
original_shape_Sphericity         0.795552  0.937018
original_shape_SurfaceArea        0.971770  0.998364
...                                    ...       ...
original_glszm_ZoneVariance       0.966698  0.974658
original_ngtdm_Busyness           0.729358  0.741922
original_ngtdm_Coarseness         0.707628  0.773980
original_ngtdm_Contrast           0.442695  0.573154
cancer                                 NaN       NaN

[62 rows x 2 columns]


/Users/veramegias/anaconda3/lib/python3.11/site-packages/pingouin/reliability.py:367: RuntimeWarning: invalid value encountered in scalar divide
  l2 = n * (msb - f2u * mse) / (f2u * (k * msj + (k * n - k - n) * mse) + n * msb)
/Users/veramegias/anaconda3/lib/python3.11/site-packages/pingouin/reliability.py:312: RuntimeWarning: invalid value encountered in scalar divide
  icc1 = (msb - msw) / (msb + (k - 1) * msw)
/Users/veramegias/anaconda3/lib/python3.11/site-packages/pingouin/reliability.py:313: RuntimeWarning: invalid value encountered in scalar divide
  icc2 = (msb - mse) / (msb + (k - 1) * mse + k * (msj - mse) / n)
/Users/veramegias/anaconda3/lib/python3.11/site-packages/pingouin/reliability.py:314: RuntimeWarning: invalid value encountered in scalar divide
  icc3 = (msb - mse) / (msb + (k - 1) * mse)
/Users/veramegias/anaconda3/lib/python3.11/site-packages/pingouin/reliability.py:315: RuntimeWarning: invalid value encountered in scalar divide
  icc1k = (msb - msw) / msb
/Users/

In [41]:
results_icc

,ICC2,ICC3
original_shape_Elongation,0.258850,0.998316
original_shape_Flatness,0.962607,0.988874
original_shape_Maximum3DDiameter,0.970904,0.999693
original_shape_Sphericity,0.795552,0.937018
original_shape_SurfaceArea,0.971770,0.998364
original_shape_SurfaceVolumeRatio,0.815889,0.880162
original_firstorder_Entropy,0.409261,0.972894
original_firstorder_InterquartileRange,0.308038,0.789293
original_firstorder_Kurtosis,0.432762,0.566926
original_firstorder_Maximum,-0.191114,-0.461449


In [38]:
results_icc.to_csv('results_icc.csv', index=False)